In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, sys

In [5]:
current_dir=os.getcwd()                     #current working directory
print(current_dir)
parent_dir=os.path.dirname(current_dir)     # Get the parent directory
print(parent_dir)
sys.path.append(parent_dir)

c:\10acadamy\week2\TellCo-Analysis\notebooks
c:\10acadamy\week2\TellCo-Analysis


In [6]:
from scripts.general_purpose import fetch_data_as_dataframe
from scripts.general_purpose import explore_dataframe

In [7]:
# Get data from PostgreSQL
# SQL query to fetch data
query = "SELECT * FROM xdr_data;"
df_tellco=fetch_data_as_dataframe(query)

In [8]:
# print dataframe columns
explore_dataframe(df_tellco)


*************** Shape: (150001, 55) ***************
**************** Info *****************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  f

In [13]:
from scripts.general_purpose import group_dataframe

# get top 10 Handset Types
top_handsets =(group_dataframe(df_tellco,'Handset Type')).value_counts().head(10)
print(top_handsets)

Handset Type
Huawei B528S-23A                19752
Apple iPhone 6S (A1688)          9419
Apple iPhone 6 (A1586)           9023
undefined                        8987
Apple iPhone 7 (A1778)           6326
Apple iPhone Se (A1723)          5187
Apple iPhone 8 (A1905)           4993
Apple iPhone Xr (A2105)          4568
Samsung Galaxy S8 (Sm-G950F)     4520
Apple iPhone X (A1901)           3813
Name: count, dtype: int64


In [14]:
top3_handset_manfacturers = (group_dataframe(df_tellco,'Handset Manufacturer')).value_counts().head(3)
print(top3_handset_manfacturers)

Handset Manufacturer
Apple      59565
Samsung    40839
Huawei     34423
Name: count, dtype: int64


###### Top 5 handsets for each of the top 3 handset manufacturers by filter the dataframe for the top manufacturers and then group the data by both the Handset Manufacturer and Handset Type columns

In [15]:
# Step 1: Get the top 3 handset manufacturers
top_manufacturers = top3_handset_manfacturers.index

# Step 2: Filter the dataframe for the top 3 manufacturers
filtered_df = df_tellco[df_tellco['Handset Manufacturer'].isin(top_manufacturers)]

# Step 3: Group by 'Handset Manufacturer' and 'Handset Type', count occurrences
top_handsets_per_manufacturer = (
    filtered_df.groupby(['Handset Manufacturer', 'Handset Type'])
    .size()
    .reset_index(name='Count')
)

# Step 4: Sort the counts within each manufacturer and get the top 5 for each
top_5_handsets = (
    top_handsets_per_manufacturer
    .sort_values(['Handset Manufacturer', 'Count'], ascending=[True, False])
    .groupby('Handset Manufacturer')
    .head(5)
)

print(top_5_handsets)


    Handset Manufacturer                    Handset Type  Count
35                 Apple         Apple iPhone 6S (A1688)   9419
31                 Apple          Apple iPhone 6 (A1586)   9023
41                 Apple          Apple iPhone 7 (A1778)   6326
55                 Apple         Apple iPhone Se (A1723)   5187
48                 Apple          Apple iPhone 8 (A1905)   4993
83                Huawei                Huawei B528S-23A  19752
104               Huawei                    Huawei E5180   2079
183               Huawei  Huawei P20 Lite Huawei Nova 3E   2021
182               Huawei                      Huawei P20   1480
224               Huawei                  Huawei Y6 2018    997
359              Samsung    Samsung Galaxy S8 (Sm-G950F)   4520
242              Samsung      Samsung Galaxy A5 Sm-A520F   3724
292              Samsung     Samsung Galaxy J5 (Sm-J530)   3696
284              Samsung     Samsung Galaxy J3 (Sm-J330)   3484
357              Samsung    Samsung Gala

##### Make a short interpretation and recommendation to marketing teams 
##### **Interpretation**:
**Top Handset Types:** The most used handset types highlight the most popular devices among users. These devices dominate user engagement, reflecting consumer preferences and technological trends.

**Top Handset Manufacturers:** The top 3 manufacturers have a significant market share, indicating strong brand loyalty and customer satisfaction within their product lines.

**Top Handsets per Manufacturer:** Specific handset models from these top manufacturers are particularly favored. These models likely drive high data consumption and user engagement, making them key targets for promotional campaigns or partnerships.

##### **Recommendations for Marketing Teams:**
**Leverage Popular Handsets:**
Target campaigns and promotions specifically for the top 10 handset types. For example, tailor advertisements, apps, or services optimized for these devices.
Use the popularity of these handsets to negotiate co-marketing deals with manufacturers.

**Partner with Top Manufacturers:**
Collaborate with the top 3 handset manufacturers to co-create targeted marketing campaigns or bundled offers.
Promote loyalty programs or exclusive deals for users of these manufacturers' devices.

**Focus on Top Handsets per Manufacturer:**
Offer special discounts or service packages tailored for users of the top 5 handsets from each manufacturer.

**Enhance Customer Engagement:**
Develop marketing materials, apps, or services optimized for the features and user bases of the top handset types and manufacturers.

##### Identify and treat all missing values and outliers in the dataset by replacing them with the mean or any possible solution of the corresponding column

In [54]:
# rows_with_nulls = df_tellco[df_tellco['MSISDN/Number'].isnull()]

# # print(rows_with_nulls)
# df_tellco.head()         # (150001, 56)

# Drop rows where any of the specified columns has NaN
df_tellco_cleaned = df_tellco.dropna(subset=['Bearer Id','MSISDN/Number','IMEI'])   # 148506 rows dropped
df_tellco_cleaned.shape

(148506, 56)

##### Task 1.1 - Your employer wants to have an overview of the users’ behavior on those applications.
###### **Aggregate per user the following information in the column**  
    . number of xDR sessions
    . Session duration
    . the total download (DL) and upload (UL) data
    . the total data volume (in Bytes) during this session for each application

In [61]:
# Perform aggregation per user (MSISDN/Number)
aggregated_data = (
    df_tellco_cleaned.groupby('MSISDN/Number')
    .agg(
        num_xdr_sessions=('Bearer Id', 'nunique'),
        total_session_duration=('Dur. (s)', 'sum'),
        total_dl_data=('Total DL (Bytes)', 'sum'),
        total_ul_data=('Total UL (Bytes)', 'sum'),
    )
    .reset_index()
)

# Calculate total data volume
aggregated_data['total_data_volume'] = (
    aggregated_data['total_dl_data'] + aggregated_data['total_ul_data']
)

print(aggregated_data)


        MSISDN/Number  num_xdr_sessions  total_session_duration  \
0        3.360100e+10                 1                 116.720   
1        3.360100e+10                 1                 181.230   
2        3.360100e+10                 1                 134.969   
3        3.360101e+10                 1                  49.878   
4        3.360101e+10                 2                  37.104   
...               ...               ...                     ...   
106466   3.379000e+10                 1                   8.810   
106467   3.379000e+10                 1                 140.988   
106468   3.197021e+12                 1                 877.385   
106469   3.370000e+14                 1                 253.030   
106470   8.823971e+14                 1                 869.844   

        total_dl_data  total_ul_data  total_data_volume  
0        8.426375e+08     36053108.0       8.786906e+08  
1        1.207552e+08     36104459.0       1.568596e+08  
2        5.566597e+08

In [58]:
# Assuming `aggregated_data` has already been created with 'total_session_duration' and 'total_data_volume'

# Step 1: Create decile classes based on total session duration
aggregated_data['decile_class'] = pd.qcut(
    aggregated_data['total_session_duration'],
    q=10,  # Divide into 10 equal parts (deciles)
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Assign decile labels
)

# Step 2: Filter for the top 5 decile classes
top_5_deciles = aggregated_data[aggregated_data['decile_class'].isin([6, 7, 8, 9, 10])]

# Step 3: Compute total data volume per decile class
total_data_per_decile = (
    top_5_deciles.groupby('decile_class')['total_data_volume']
    .sum()
    .reset_index()
    .sort_values(by='decile_class')
)

# Display results
print(total_data_per_decile)

  decile_class  total_data_volume
0            6       6.405460e+12
1            7       6.125652e+12
2            8       7.806072e+12
3            9       9.286285e+12
4           10       1.483104e+13
